In [ ]:
# %%
import os
import pandapower as pp
import pandas as pd
import polars as pl
from polars import col as c

from pipelines.dig_a_plan import DigAPlan
from polars import selectors as cs 
# from pipelines.dig_a_plan_d_model import DigAPlan
from data_display.grid_plotting import plot_grid_from_pandapower
from data_display.output_processing import compare_dig_a_plan_with_pandapower
from data_connector import pandapower_to_dig_a_plan_schema
from general_function import pl_to_dict

from pyomo_utility import extract_optimization_results

import matplotlib.pyplot as plt

os.chdir(os.getcwd().replace("/src", ""))
os.environ['GRB_LICENSE_FILE'] = os.environ["HOME"] + "/gurobi_license/gurobi.lic"


In [2]:
LOAD_FACTOR  = 1
TEST_CONFIG = [
    {"line_list": [], "switch_list":  []},
    {"line_list": [6, 9], "switch_list":  [25, 28]},
    {"line_list": [2, 6, 9], "switch_list":  [21, 25, 28]},
    {"line_list": [16], "switch_list":  [35]},
    {"line_list": [1], "switch_list":  [20]},
    {"line_list": [10], "switch_list":  [29]},
    {"line_list": [7, 11], "switch_list":  [26, 30]}
]
NB_TEST = 0
# set input data

net = pp.from_pickle(".cache/input_data/mv_example.p")
combination_results = pl.read_csv(".cache/input_data/load_facotr_1_results.csv")\
    .with_columns(
        pl.concat_list(cs.starts_with("switch")).alias("switch_list")
    )

net["load"]["p_mw"] = net["load"]["p_mw"]*LOAD_FACTOR
net["load"]["q_mvar"] = net["load"]["q_mvar"]*LOAD_FACTOR


net["line"].loc[:, "max_i_ka"] = 1
net["line"].loc[TEST_CONFIG[NB_TEST]["line_list"], "max_i_ka"] = 1e-2

net["line"]["c_nf_per_km"] = 0


In [3]:

base_grid_data = pandapower_to_dig_a_plan_schema(net)


In [ ]:
dig_a_plan: DigAPlan = DigAPlan(verbose= False, big_m = 1e2)

dig_a_plan.add_grid_data(**base_grid_data)
dig_a_plan.solve_models_pipeline(max_iters = 10000)

2025-07-08 08:04:12 B01BPC15 pipelines.dig_a_plan[1382] WARNING The resulting graph considering normal switch is NOT a tree.
 The initial state of switches is determined solving master model.


In [ ]:
dig_a_plan.master_model_instance.optimality_cut.pprint()

In [ ]:
dig_a_plan.master_model_instance.infeasibility_cut.pprint()

In [ ]:
dig_a_plan.master_model_instance.objective() 

In [ ]:
extract_optimization_results(dig_a_plan.infeasible_slave_model_instance, "slack_i_sq")\
    .filter(c("slack_i_sq") >1e-5)

In [ ]:
print(dig_a_plan.marginal_cost.to_pandas().to_string())

In [ ]:
print(dig_a_plan.marginal_cost.to_pandas().to_string())
print(dig_a_plan.extract_switch_status().to_pandas().to_string())

In [ ]:
# node_data, edge_data =  compare_dig_a_plan_with_pandapower(dig_a_plan=dig_a_plan, net=net)

In [ ]:
plot_grid_from_pandapower(net=net, dig_a_plan=dig_a_plan)

In [ ]:
if TEST_CONFIG[NB_TEST]["switch_list"]:

    optimum = combination_results.filter(
        pl.all_horizontal(
            c("switch_list").list.contains(switch_id)
            for switch_id in TEST_CONFIG[NB_TEST]["switch_list"]
        )
    ).sort("slave_results").to_dicts()[0]
else:
    optimum = combination_results.sort("slave_results").to_dicts()[0]
reslut_switch_id = dig_a_plan.extract_switch_status().filter(c("open")).sort("edge_id")["edge_id"].to_list()

print(f"Slave results: {dig_a_plan.slave_obj}, switches: {reslut_switch_id}")
print(f"Slave optimum: {2*optimum["slave_results"]}, switches: {optimum["switch_list"]}")

In [ ]:
combination_results.filter(
        pl.all_horizontal(
            c("switch_list").list.contains(switch_id)
            for switch_id in [30, 26]
        )
    ).sort("slave_results").to_dicts()[0]

In [ ]:
open_switches_list = extract_optimization_results(dig_a_plan.master_model_instance, "delta").filter(c("delta") == 0.0)["S"].to_list()

In [ ]:
dig_a_plan_test: DigAPlanTest = DigAPlanTest(
    verbose= False, big_m = 1e6, penalty_cost = 1e3, current_factor= 1e2, voltage_factor = 1, power_factor = 1,
)
dig_a_plan_test.add_grid_data(**base_grid_data)

dig_a_plan_test.test_one_switch_combinations(open_switches_list=open_switches_list)

In [ ]:
print(dig_a_plan_test.slack_v_neg.to_pandas().to_string())

In [ ]:
print(extract_optimization_results(dig_a_plan_test.slave_model_instance, "master_d").to_pandas().to_string())

In [ ]:
print(extract_optimization_results(dig_a_plan_test.slave_model_instance, "i_sq").to_pandas().to_string())

In [ ]:
print(dig_a_plan_test.slack_i_sq.to_pandas().to_string())